<html>
<head>
<link href="https://fonts.googleapis.com/css2?family=Nunito:wght@400;700&display=swap" rel="stylesheet">
<style>
h1 {
    font-weight: bold
}
h1, p, ul {
    font-family: saxMono
}
p {
    font-size: 16px;
    text-align: justify;
    padding: 0 20px 0 0;
}
</style>
</head>
<body>
<h1>Case Study: KDD Cup 2009</h1>
<p>A Administração de Relacionamento com o Consumidor é um elemento-chave das estratégias modernas de marketing. A maneira mais prática de adquirir conhecimento sobre consumidores é produzindo pontuações. Uma pontuação é a saída de um modelo e serve como avaliação de todas as instâncias de uma variável-alvo que se pretenda explicar. Ferramentas que produzem pontuações permitem projetar informações quantificáveis em uma dada população. A pontuação é calculada usando variáveis de entrada que descrevem as instâncias. As pontuações são então usadas pelo sistema de informação, por exemplo, para personalizar o relacionamento com o consumidor.
</p>
<p>
Em 2009, a conferência Knowledge Discovery and Data Mining (KDD) lançou um desafio envolvendo aprendizagem de máquina. Ela disponibilizou dados de consumidores da empresa de telecomunicações francesa Orange, que deveriam ser utilizados para a construção de um modelo capaz de prever três comportamentos do consumidor. <b>A planilha fornecida tem 50000 instâncias e 230 variáveis, sendo as primeiras 190 variáveis numéricas e as demais 40 categóricas</b>. As variáveis de saída do modelo eram
</p>
<ul>
<li> o cancelamento da conta ('churn');
<li> a tendência de usar novos produtos e serviços ('appetency';
<li> e a propensão para comprar upgrades ou adicionais com maior margem de lucro quando apresentados (upselling),
</ul>
<p>
fornecidos de forma binária (-1 para não, 1 para sim).
</p>
<p>
O trabalho apresentado aqui envolve a criação de um modelo para predição de churn, appetency e upselling com base nos dados fornecidos dos clientes da Orange. <b>O critério para avaliação do modelo será a métrica AUC</b>, que dimensiona o trade-off entre taxa de falsos positivos e taxa de verdadeiros positivos: quanto mais próximo de 1, menor o trade-off e melhor o modelo.
</p>
<p>
Detalhes sobre a competição: https://www.kdd.org/kdd-cup/view/kdd-cup-2009
</p>
</body>
</html>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# We don't know exactly what is the meaning of each feature, but we can try to find out.
# First we have to load the data and the outcome.
features = pd.read_csv('kdd/orange_small_train.data', sep='\t', na_filter = False)
outcome  = pd.read_csv('kdd/orange_small_train_churn.labels', header=None)

In [3]:
print(features.shape)   # We got 50000 samples of sets of 230 features
print(outcome.shape)    # We got 50000 results

(50000, 230)
(50000, 1)


In [10]:
# We can find out what are the possible outcomes of the model by using the np.unique() function.
print(np.unique(outcome, return_counts=True))

(array([-1,  1]), array([46328,  3672]))


In [11]:
# We can se we have only two outcomes: -1 and 1.
# And that the proportion is not balanced.
# Let's look at the features.
features.head()

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,...,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
0,,,,,,1526,7,,,,...,oslk,fXVEsaq,jySVZNlOJy,,,xb3V,RAYp,F2FyR07IdsN7I,,
1,,,,,,525,0,,,,...,oslk,2Kb5FSF,LM8l689qOp,,,fKCe,RAYp,F2FyR07IdsN7I,,
2,,,,,,5236,7,,,,...,Al6ZaUT,NKv4yOc,jySVZNlOJy,,kG3k,Qu4f,02N6s8f,ib5G6X1eUxUn6,am7c,
3,,,,,,,0,,,,...,oslk,CE7uk3u,LM8l689qOp,,,FSa2,RAYp,F2FyR07IdsN7I,,
4,,,,,,1029,7,,,,...,oslk,1J2cvxe,LM8l689qOp,,kG3k,FSa2,RAYp,F2FyR07IdsN7I,mj86,


In [12]:
# Ow, my fucking god! What is that? oslk? 7, 0?
# Well, let's understand it a little bit more. 
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 230 entries, Var1 to Var230
dtypes: float64(2), int64(1), object(227)
memory usage: 87.7+ MB


In [ ]:
# We know we have 230 entries, from Var1 to Var230.
# We know, from the challange description we have 190 numbers features, from Var1 to Var190.
# We know, from the challange description we have 40 categorical features, from Var191 to Var230.
# We know we have 1 target, from target.
# We know there are 2 float64 features, 1 int64 feature and 227 object features.
# And that's it. We know we have a mess of data, and we need to work hard on it.

In [13]:
# Let's begin:
# Get all the features names and store it in a numpy array
all_vars = np.array(features.columns)
print(all_vars)

['Var1' 'Var2' 'Var3' 'Var4' 'Var5' 'Var6' 'Var7' 'Var8' 'Var9' 'Var10'
 'Var11' 'Var12' 'Var13' 'Var14' 'Var15' 'Var16' 'Var17' 'Var18' 'Var19'
 'Var20' 'Var21' 'Var22' 'Var23' 'Var24' 'Var25' 'Var26' 'Var27' 'Var28'
 'Var29' 'Var30' 'Var31' 'Var32' 'Var33' 'Var34' 'Var35' 'Var36' 'Var37'
 'Var38' 'Var39' 'Var40' 'Var41' 'Var42' 'Var43' 'Var44' 'Var45' 'Var46'
 'Var47' 'Var48' 'Var49' 'Var50' 'Var51' 'Var52' 'Var53' 'Var54' 'Var55'
 'Var56' 'Var57' 'Var58' 'Var59' 'Var60' 'Var61' 'Var62' 'Var63' 'Var64'
 'Var65' 'Var66' 'Var67' 'Var68' 'Var69' 'Var70' 'Var71' 'Var72' 'Var73'
 'Var74' 'Var75' 'Var76' 'Var77' 'Var78' 'Var79' 'Var80' 'Var81' 'Var82'
 'Var83' 'Var84' 'Var85' 'Var86' 'Var87' 'Var88' 'Var89' 'Var90' 'Var91'
 'Var92' 'Var93' 'Var94' 'Var95' 'Var96' 'Var97' 'Var98' 'Var99' 'Var100'
 'Var101' 'Var102' 'Var103' 'Var104' 'Var105' 'Var106' 'Var107' 'Var108'
 'Var109' 'Var110' 'Var111' 'Var112' 'Var113' 'Var114' 'Var115' 'Var116'
 'Var117' 'Var118' 'Var119' 'Var120' 'Var121' 'Var1

In [14]:
# This information is gotten from the description of the challenge
num_vars = np.array(all_vars[:190])
cat_vars = np.array(all_vars[190:])

In [19]:
# We can know how many times each outcome of each feature appears in the dataset
# by using the value_counts() method of a DataFrame column.
print(np.unique(features['Var10']))
features.groupby('Var10').size().sort_values(ascending=False).head(20)

['' '0' '1.232559e+007' '100521' '1007739' '1025793' '1030095' '10314'
 '103203' '1035144' '1035963' '103761' '1037925' '10431' '1045386' '10548'
 '105795' '107343' '1076886' '108054' '109386' '109692' '110556' '11106'
 '1114848' '111627' '112617' '1127484' '113202' '1136295' '11367'
 '1142271' '1143171' '1145637' '1148094' '1166535' '116730' '1168704'
 '117351' '1175994' '117774' '118656' '1188801' '11916' '1192329'
 '1194975' '119727' '1201887' '120690' '120897' '1210158' '1212660'
 '12159' '1236753' '1255806' '125973' '126207' '1263465' '127224'
 '1310013' '1326897' '1332009' '1341216' '1349892' '1350' '1351548'
 '1361808' '1365948' '138699' '138762' '13905' '139653' '141129' '142236'
 '1425618' '1425627' '1425681' '14319' '144117' '144990' '1459863'
 '1472814' '1473831' '1481175' '1487079' '1489365' '1495917' '1501101'
 '150183' '1503' '1507176' '151074' '15138' '1523565' '152496' '152901'
 '1532070' '153414' '1543662' '1547820' '1549305' '1555110' '1555200'
 '1555218' '1555902' '1

Var10
           48513
0            902
777600        17
1555200       14
2332800       10
3110400        6
5443200        3
1425618        3
3888000        3
265230         2
4665600        2
2709           2
532035         1
522639         1
45342          1
452115         1
4491           1
4477248        1
446328         1
44523          1
dtype: int64

In [ ]:
# Can you notice there are so few outcomes with a significant frequency?
# From all the 48513 points, only 11 have more than 1 occurrence.
# This is a problem for the model, because it will be difficult to learn from so few examples.
# We will need to do something about it.

In [21]:
# First, let's understand all the features. We can do the same bofore using a for loop.
counts_per_column = pd.DataFrame()
for col in num_vars:
  col_count = features.groupby(col).size()
  counts_per_column = counts_per_column.append(col_count, ignore_index = True)

/tmp/ipykernel_1367/437899381.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  counts_per_column = counts_per_column.append(col_count, ignore_index = True)
/tmp/ipykernel_1367/437899381.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  counts_per_column = counts_per_column.append(col_count, ignore_index = True)
/tmp/ipykernel_1367/437899381.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  counts_per_column = counts_per_column.append(col_count, ignore_index = True)
/tmp/ipykernel_1367/437899381.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  counts_per_column = counts_per_column.append(col_count, ignore_index = True)
/tmp/ipy

In [ ]:
counts_per_column.shape

In [ ]:
# Now we are going to 